In [ ]:
!pip install ultralytics opencv-python matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from google.colab import files

In [ ]:
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

In [ ]:
model = YOLO("yolov8x-seg.pt")

In [ ]:
results = model(image_path)

In [ ]:
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
mask = np.zeros(img.shape[:2], dtype=np.uint8)

for r in results:
    if r.masks is None:
        continue
    for box, cls, seg in zip(r.boxes.xyxy, r.boxes.cls, r.masks.xy):
        if int(cls) == 0:  # class 0 = person in COCO dataset
            seg = np.array(seg, dtype=np.int32)
            cv2.fillPoly(mask, [seg], 255)

In [ ]:
blurred = cv2.GaussianBlur(img, (51, 51), 30)
final = img.copy()
final[mask == 255] = blurred[mask == 255]

final_rgb = cv2.cvtColor(final, cv2.COLOR_BGR2RGB)


In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.title("Original Image")
plt.imshow(img_rgb)
plt.axis("off")

plt.subplot(1,2,2)
plt.title("Humans Blurred")
plt.imshow(final_rgb)
plt.axis("off")

plt.show()